<a href="https://colab.research.google.com/github/maxpestr/cozyharbour/blob/main/CatBoostFut252_Vol_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from catboost import Pool
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
dataframe = pd.read_csv('/content/drive/MyDrive/FinMarkt/futures_nodate.csv')

In [ ]:
def getdataset(token):
  data = pd.read_csv('/content/drive/MyDrive/FinMarkt/futures_nodate.csv')

  # Вычисление изменений за 2 дня в процентах
  data2 = ((data - data.shift(2)) / data.shift(2)) * 100

  # Вычисление изменений за 5 дней в процентах
  data5 = ((data - data.shift(5)) / data.shift(5)) * 100

  data22 = ((data - data.shift(22)) / data.shift(22)) * 100

  data252 = ((data - data.shift(252)) / data.shift(252)) * 100

  token_change = token + '_Change'

  data[token_change] = data[token].shift(-1).ge(data[token]).replace({True: 1, False: -1})
  data[token_change].iloc[-1] = None

  data.loc[:, data.columns != token_change] = data.loc[:, data.columns != token_change].pct_change() * 100

  data_vol_5 = data.loc[:, data.columns != token_change].rolling(window=5).std()
  data_vol_22 = data.loc[:, data.columns != token_change].rolling(window=22).std()

  # Добавление суффиксов к столбцам
  data2.columns = [f"{col}_2" for col in data2.columns]
  data5.columns = [f"{col}_5" for col in data5.columns]
  data22.columns = [f"{col}_22" for col in data22.columns]
  data252.columns = [f"{col}_252" for col in data252.columns]
  data_vol_5.columns = [f"{col}_Vol5" for col in data_vol_5.columns]
  data_vol_22.columns = [f"{col}_Vol22" for col in data_vol_22.columns]

  data = pd.concat([data_vol_22, data_vol_5, data252, data22, data5, data2, data], axis=1)
  #data = pd.concat([data22, data5, data2, data], axis=1)

  data = data.dropna()

  return data

In [ ]:
data = getdataset('HG')

<ipython-input-20-c26bdb34c026>:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data[token_change] = data[token].shift(-1).ge(data[token]).replace({True: 1, False: -1})
<ipython-input-20-c26bdb34c026>:17: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` i

In [ ]:
data

,CL_Vol22,DX_Vol22,ES_Vol22,GC_Vol22,HG_Vol22,US_Vol22,VX_Vol22,ZT_Vol22,CL_Vol5,DX_Vol5,...,ZT_2,CL,DX,ES,GC,HG,US,VX,ZT,HG_Change
252,1.760636,0.424234,0.955445,0.637755,1.773301,1.100346,0.425724,0.111689,1.636953,0.334174,...,-0.082728,-2.322978,0.309231,-0.232313,-0.214448,0.577478,-0.036860,0.024156,0.015056,1.0
253,1.810310,0.450105,0.951937,0.649938,1.719457,1.046615,0.425905,0.109443,1.710079,0.533320,...,0.082809,2.034384,-0.905155,0.529213,0.839490,1.572112,0.036873,-0.338099,0.067743,-1.0
254,1.829952,0.443152,0.955390,0.658565,1.753037,1.045354,0.416038,0.110914,2.100126,0.537350,...,0.188175,-3.313676,0.350808,-0.821226,-0.599401,-2.866756,0.405455,0.218087,0.120351,-1.0
255,1.852022,0.472544,0.920725,0.666049,1.755006,1.033404,0.406353,0.110663,2.086292,0.617909,...,0.180526,-2.149288,0.712354,-0.891720,-0.871022,-1.704309,0.513950,0.193433,0.060103,1.0
256,1.763924,0.456718,0.926499,0.673326,1.772609,1.045338,0.400638,0.112281,2.147741,0.622111,...,-0.022538,-0.029682,-0.150632,-0.471294,-0.331193,1.085424,0.986121,0.000000,-0.082591,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,2.252644,0.449186,1.231580,0.970337,1.527343,0.780190,14.249024,0.165696,2.201250,0.570752,...,0.048896,-1.346535,0.552404,-0.571429,-0.713441,-1.962142,-0.125628,6.752412,-0.015035,-1.0
3672,2.266386,0.457400,1.223519,0.956317,1.530609,0.788149,14.283258,0.167033,2.233388,0.554088,...,-0.078935,1.726214,0.282108,-0.022276,0.468458,-0.400283,-0.377358,-3.915663,-0.063910,-1.0
3673,2.158971,0.458869,1.194599,0.908012,1.448234,0.783821,14.281942,0.164519,2.564206,0.315369,...,-0.120301,-3.235991,0.296121,0.855577,-0.742877,-0.248227,-0.707071,-3.448276,-0.056427,-1.0
3674,2.311522,0.450870,1.240965,0.911845,1.486533,0.798304,14.967647,0.160817,2.306599,0.318316,...,0.033856,-4.363785,0.108257,-2.032431,-0.457821,-3.080934,1.093591,23.701299,0.090334,-1.0


In [ ]:
data['HG_Change'].value_counts()

,count
HG_Change,
1.0,1737
-1.0,1687


In [ ]:
1737/(1737+1687)

0.5073014018691588

In [ ]:
X = data.drop('HG_Change', axis=1)  # Все столбцы, кроме целевого
y = data['HG_Change']  # Целевой столбец

In [ ]:
X_train = X.iloc[:3300]
X_test = X.iloc[3300:]
y_train = y.iloc[:3300]
y_test = y.iloc[3300:]

In [ ]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=8)

In [ ]:
# Обучение модели
model.fit(X_train, y_train)

0:	learn: 0.6846231	total: 157ms	remaining: 2m 36s
1:	learn: 0.6786567	total: 248ms	remaining: 2m 3s
2:	learn: 0.6713798	total: 351ms	remaining: 1m 56s
3:	learn: 0.6651185	total: 449ms	remaining: 1m 51s
4:	learn: 0.6584828	total: 548ms	remaining: 1m 48s
5:	learn: 0.6530236	total: 621ms	remaining: 1m 42s
6:	learn: 0.6464586	total: 707ms	remaining: 1m 40s
7:	learn: 0.6408350	total: 795ms	remaining: 1m 38s
8:	learn: 0.6378533	total: 892ms	remaining: 1m 38s
9:	learn: 0.6323204	total: 994ms	remaining: 1m 38s
10:	learn: 0.6274469	total: 1.09s	remaining: 1m 37s
11:	learn: 0.6218925	total: 1.19s	remaining: 1m 37s
12:	learn: 0.6159893	total: 1.29s	remaining: 1m 37s
13:	learn: 0.6119414	total: 1.39s	remaining: 1m 37s
14:	learn: 0.6077419	total: 1.48s	remaining: 1m 37s
15:	learn: 0.6020882	total: 1.58s	remaining: 1m 37s
16:	learn: 0.5975914	total: 1.68s	remaining: 1m 37s
17:	learn: 0.5933564	total: 1.78s	remaining: 1m 37s
18:	learn: 0.5880602	total: 1.88s	remaining: 1m 36s
19:	learn: 0.5830869	to

In [ ]:
# Прогнозирование
y_pred = model.predict(X_test)

# Оценка точности
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.54
